In [27]:
from app.models import Comment

In [28]:
class PostCommentPathDetails:
    """" An object containing comment path details for a post """
    def __init__(self, post_id):
        self.id = post_id

    @property
    def full_comment_tree_list(self):
        return self._get_comment_tree_list()

    @property
    def comment_tree_list(self):
        return self._get_pruned_list(self.full_comment_tree_list)

    @property
    def levels(self):
        return self._get_levels()

    def _get_path(self, comment_id, path=None):
        """ Gets a comment's tree path from its ID """
        comment = Comment.query.filter_by(id=comment_id).first()
        if path is None:  # Create path list on first pass
            path = list()
        if comment.id not in path:  # Add initial comment_id to path
            path.append(comment.id)
        if comment.parent_id:  # Add parent_id to path
            path.append(comment.parent_id)
            return self._get_path(comment.parent_id, path=path)  # Loop
        else:
            path.reverse()  # Flip list so it begins at root
            return path

    def _get_comment_tree_list(self):
        """ Returns a full list of comment tree paths """
        comments = Comment.query.filter_by(post_id=self.id).all()
        comment_tree_list = list()
        for comment in comments:
            comment_path = self._get_path(comment.id)
            comment_tree_list.append(comment_path)
        return comment_tree_list

    def _check_sublist(self, sublist, full_list):
        """
        Checks if comment tree exists as a subset 
        of any other comment tree on the post
        """
        check = False
        for item in full_list:
            if set(sublist).issubset(set(item)):
                check = True
                return check
            else:
                pass

        return check

    def _get_pruned_list(self, input_tree_list, pruned_list=None):
        """ Removes redundant paths from list of comment tree paths """
        tree_list = input_tree_list.copy()  # Copy tree list to work with
        if pruned_list == None:  # Create pruned path list on first pass
            pruned_list = list()
        if len(tree_list) > 0:
            for tree in tree_list:
                tree_list.remove(tree)
                # Check if tree is part of a larger, unique tree in the list
                if not self._check_sublist(tree, tree_list):
                    pruned_list.append(tree)  # Append full unique tree
                    return self._get_pruned_list(  # Loop
                        tree_list, pruned_list=pruned_list)
                else:
                    return self._get_pruned_list(  # Loop
                        tree_list, pruned_list=pruned_list)
        else:  # Return when list has been exhausted
            return pruned_list

    def _get_levels(self):
        """ Returns the level count of the post's comments """
        return len(max(self.comment_tree_list, key=len))

    def get_depth_index(self, comment_id):
        """ Returns the depth (level index) of the comment """
        return (len(self._get_path(comment_id)) - 1)

    def get_level(self, n, parent_id=None, root_id=None):
        """
        Gets all comments at level 'n'.

        Adding the 'parent_id=<PARENT-COMMENT-ID>' argurment will get only
            comments stemming from that parent at level 'n'.

        Adding the 'root_id=<ROOT-COMMENT-ID>' argument will get only
            comments at level 'n' from the root comment.
        """
        level = list()
        for i in self.comment_tree_list:
            # Parent ID argument check
            if parent_id and i[parent_id] \
                and len(i) >= (n + 1) \
                and i[n] not in level:  # Ignore duplicates
                level.append(i[n])
            # Root ID argument check
            if root_id and i[0] == root_id and len(i) >= (n + 1) \
                and i[n] not in level:  # Ignore duplicates
                level.append(i[n])
            # No argument check
            elif root_id is None and len(i) >= (n + 1) \
                and i[n] not in level:  # Ignore duplicates
                level.append(i[n])
            else:
                pass
        if len(level) >= 1:
            return level
        else:
            return None
    
    def get_comment_trees(self, id):
        # Get all comment trees for a comment
        trees = []
        for tree in self.comment_tree_list:
            if id in tree:
                trees.append(tree)
        if len(trees) > 0:
            return trees
        else:
            return None
    
    def get_children(self, comment_id, paths=False):
        """ Returns a list of a comment's children or full child paths """
        child_depth = self.get_depth_index(comment_id) + 1
        children = list()
        for i in self.comment_tree_list:
            if comment_id in i and not paths and child_depth < len(i):  # Child items only
                if i[child_depth] not in children:  # Remove duplicates
                    children.append(i[child_depth])
                else:
                    pass
            elif comment_id in i and paths and child_depth < len(i):  # Full child paths
                child_path = i[child_depth::]
                children.append(child_path)
            else:
                pass
        if len(children) > 0:
            return children
        else:
            return None
    
    def get_parent(self, comment_id):
        comment = Comment.query.filter_by(id=comment_id).first()
        return comment.parent_id
    
    def _make_path(self, id, tree, map_path=None, index=0):
        while index in range(tree.index(id)):
            node_idx = self._get_list_index_for_node(tree[index], map_path)
            index += 1
            return self._make_path(id, tree, map_path=map_path[node_idx]['children'], index=index)

        return map_path
    
    def _check_node_id(self, node, id):
        """ Checks if a node has the specified ID """
        for key, val in node.items():
            if val == id:
                return True
            else:
                return False
    
    def _check_nodes_in_list(self, id, lst):
        """ Checks if a node already exists in a list """
        for node_dict in lst:
            if self._check_node_id(node_dict, id):
                return True
            else:
                return False

    def _find_node_route_and_attach(self, node_id, node_map=None, tree=None, node_payload=None, idx=None):
        """ Finds a node in the map """
        if not idx:
            idx = 0
        item = tree[idx]

        if item == node_id:
            for dict_node in node_map:
                if self._check_node_id(dict_node, node_id):
                    endpoint = dict_node['children']
                    endpoint.append(node_payload)
        else:
            for dict_node in node_map:
                if self._check_node_id(dict_node, item):
                    route = dict_node['children']
                    idx += 1
                    return self._find_node_route_and_attach(
                        node_id, node_map=route, tree=tree, node_payload=node_payload, idx=idx)
    
    def _make_tree_in_map(self, tree, map=None, idx=None):
        if not map:
            map = list()
        if not idx:
            idx = 0

        if idx <= (len(tree) - 1):
            item = tree[idx]
            if not self._check_nodes_in_list(item, map):
                node = self._node(item)
                map.append(node)
                dict_node = self._get_list_index_for_node(item, map)
                map_update = map[dict_node]['children']
                idx += 1
                return self._make_tree_in_map(tree, map=map_update, idx=idx)
            else:
                dict_node = self._get_list_index_for_node(item, map)
                map_update = map[dict_node]['children']
                idx += 1
                return self._make_tree_in_map(tree, map=map_update, idx=idx)
        else:
            return map

    def _get_list_index_for_node(self, id, lst):
        for node_dict in lst:
            if self._check_node_id(node_dict, id):
                return lst.index(node_dict)
            else:
                pass

    def _append_list_in_dict(self, key_name, value, target):
        """
        Appends the list of a specified key, 'key_name', 
        with 'value' in the 'target' dict
        """
        for key, val in target.items():
                if key == key_name:
                    return val.append(value)
    
    def insert_node(self, parent_id, node, map):
        for node_dict in map:
            if self._check_node_id(node_dict, parent_id):
                return self._append_list_in_dict('children', node, node_dict)
    
    def _node(self, id, children=[]):
        node = {'id': id, 'children': children}
        return node
        
    def _make_child_nodes(self, id, child_node=None):
        """
        Creates list containing child node dicts
        Returns an empty list if reference node has no children
        """
        level_index = self.get_depth_index(id)
        if not child_node:
            child_node = list()
        if level_index == 0:
            child_node.append(self._node(id))
        children = self.get_children(id)  # Get child comment IDs
        if children:
            for child in children:
                cnode_children = self.get_children(child)  # Get child comment's children
                if cnode_children:
                    for c in cnode_children:
                        c_children = self._make_child_nodes(c, child_node=child_node)
                        child_node.append(self._node(child, children=c_children))
                else:
                    child_node.append(self._node(child))
            return child_node
        else:
            return []

    def _make_nodes_in_tree(self, tree):
        nodes = list()
        for item in tree:
            if tree.index(item) == 0:
                node = self._node(item)
                nodes.append(node)
            else:
                path = self._make_path(item, tree, map_path=nodes)
                node_path = nodes + path
                node = self._node(item)
                parent = self.get_parent(item)
                self.insert_node(parent, node, node_path)

        return nodes
    
    # def _make_nodes_in_tree_2(self, tree):
    #     nodes = list()
    #     for item in tree:
    #         i_node = self._node(item)
    #         parent = self.get_parent(item)
    #         if not parent:
    #             if not self._check_nodes_in_list(item, nodes):
    #                 nodes.append(i_node)
    #         else:
    #             path = self._make_path(self, item, tree, map_path=nodes)
    #             if not self._check_nodes_in_list(parent, path):


In [29]:
pcpd = PostCommentPathDetails(1)

Return all comment IDs by given depth level index

In [30]:
print(f'LEVEL 0: {pcpd.get_level(0)} | LEVEL 1: {pcpd.get_level(1)} | LEVEL 2: {pcpd.get_level(2)}')

LEVEL 0: [1, 10] | LEVEL 1: [4, 3, 2] | LEVEL 2: [5, 6, 7, 8, 9]


Return comment's depth index

In [31]:
pcpd.get_depth_index(4)

1

Get comment's direct children

In [32]:
pcpd.get_children(4)

[5, 6, 7]

Get comment's childrens' paths

In [33]:
pcpd.get_children(1, paths=True)

[[4, 5], [4, 6], [4, 7], [3, 8], [2, 9]]

Return every full comment path

In [34]:
pcpd.comment_tree_list

[[1, 4, 5], [1, 4, 6], [1, 4, 7], [1, 3, 8], [1, 2, 9], [10]]

In [35]:
xlst = [1, 4, 5]
mymap = pcpd._make_tree_in_map(xlst)

In [36]:
print(mymap)

[]
